In [1]:
## Loading Library
library(readr)
library(class)
install.packages('gmodels')
library(gmodels)
library(dplyr)

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

also installing the dependencies ‘gtools’, ‘gdata’



Attaching package: ‘dplyr’


The following objects are masked from ‘package:stats’:

    filter, lag


The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union




In [2]:
# Load the breast cancer dataset
install.packages('mlbench')
data("BreastCancer", package = "mlbench")

# Assign the dataset to a variable
bc_data <- BreastCancer
head(bc_data)

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)



,Id,Cl.thickness,Cell.size,Cell.shape,Marg.adhesion,Epith.c.size,Bare.nuclei,Bl.cromatin,Normal.nucleoli,Mitoses,Class
,<chr>,<ord>,<ord>,<ord>,<ord>,<ord>,<fct>,<fct>,<fct>,<fct>,<fct>
1,1000025,5,1,1,1,2,1,3,1,1,benign
2,1002945,5,4,4,5,7,10,3,2,1,benign
3,1015425,3,1,1,1,2,2,3,1,1,benign
4,1016277,6,8,8,1,3,4,3,7,1,benign
5,1017023,4,1,1,3,2,1,3,1,1,benign
6,1017122,8,10,10,8,7,10,9,7,1,malignant


In [3]:
# Data Exploration
table(bc_data$Class)
round(prop.table(table(bc_data$Class)) * 100, digits = 1)

sum(is.na(bc_data))
bc_data_clean<-na.omit(bc_data)

sum(is.na(bc_data_clean))

summary(bc_data_clean)


   benign malignant 
      458       241 


   benign malignant 
     65.5      34.5 

[1] 16

[1] 0

      Id             Cl.thickness   Cell.size     Cell.shape  Marg.adhesion
 Length:683         1      :139   1      :373   1      :346   1      :393  
 Class :character   5      :128   10     : 67   2      : 58   2      : 58  
 Mode  :character   3      :104   3      : 52   10     : 58   3      : 58  
                    4      : 79   2      : 45   3      : 53   10     : 55  
                    10     : 69   4      : 38   4      : 43   4      : 33  
                    2      : 50   5      : 30   5      : 32   8      : 25  
                    (Other):114   (Other): 78   (Other): 93   (Other): 61  
  Epith.c.size  Bare.nuclei   Bl.cromatin  Normal.nucleoli    Mitoses   
 2      :376   1      :402   3      :161   1      :432     1      :563  
 3      : 71   10     :132   2      :160   10     : 60     2      : 35  
 4      : 48   2      : 30   1      :150   3      : 42     3      : 33  
 1      : 44   5      : 30   7      : 71   2      : 36     10     : 14  
 6      : 40   3      : 28 

In [4]:
bc_data_clean<-select(bc_data_clean,-Id)
colnames(bc_data_clean)[colnames(bc_data_clean) == "Class"] <- "diagnosis"
head(bc_data_clean)

,Cl.thickness,Cell.size,Cell.shape,Marg.adhesion,Epith.c.size,Bare.nuclei,Bl.cromatin,Normal.nucleoli,Mitoses,diagnosis
,<ord>,<ord>,<ord>,<ord>,<ord>,<fct>,<fct>,<fct>,<fct>,<fct>
1,5,1,1,1,2,1,3,1,1,benign
2,5,4,4,5,7,10,3,2,1,benign
3,3,1,1,1,2,2,3,1,1,benign
4,6,8,8,1,3,4,3,7,1,benign
5,4,1,1,3,2,1,3,1,1,benign
6,8,10,10,8,7,10,9,7,1,malignant


In [6]:
bc_data_clean$Bare.nuclei<-as.numeric(bc_data_clean$Bare.nuclei)
bc_data_clean$Bl.cromatin<-as.numeric(bc_data_clean$Bl.cromatin)
bc_data_clean$Cell.shape<-as.numeric(bc_data_clean$Cell.shape)
bc_data_clean$Cell.size<-as.numeric(bc_data_clean$Cell.size)
bc_data_clean$Cl.thickness<-as.numeric(bc_data_clean$Cl.thickness)
bc_data_clean$Epith.c.size<-as.numeric(bc_data_clean$Epith.c.size)
bc_data_clean$Marg.adhesion<-as.numeric(bc_data_clean$Marg.adhesion)
bc_data_clean$Mitoses<-as.numeric(bc_data_clean$Mitoses)
bc_data_clean$Normal.nucleoli<-as.numeric(bc_data_clean$Normal.nucleoli)

In [7]:
head(bc_data_clean)

,Cl.thickness,Cell.size,Cell.shape,Marg.adhesion,Epith.c.size,Bare.nuclei,Bl.cromatin,Normal.nucleoli,Mitoses,diagnosis
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<fct>
1,5,1,1,1,2,1,3,1,1,benign
2,5,4,4,5,7,10,3,2,1,benign
3,3,1,1,1,2,2,3,1,1,benign
4,6,8,8,1,3,4,3,7,1,benign
5,4,1,1,3,2,1,3,1,1,benign
6,8,10,10,8,7,10,9,7,1,malignant


In [8]:
# Data transformation
normalize <- function(x) {
  return ((x - min(x)) / (max(x) - min(x)))
}
new_bcd <- as.data.frame(lapply(select(bc_data_clean,-diagnosis), normalize))

In [9]:
head(new_bcd)

,Cl.thickness,Cell.size,Cell.shape,Marg.adhesion,Epith.c.size,Bare.nuclei,Bl.cromatin,Normal.nucleoli,Mitoses
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,0.4444444,0.0000000,0.0000000,0.0000000,0.1111111,0.0000000,0.2222222,0.0000000,0
2,0.4444444,0.3333333,0.3333333,0.4444444,0.6666667,1.0000000,0.2222222,0.1111111,0
3,0.2222222,0.0000000,0.0000000,0.0000000,0.1111111,0.1111111,0.2222222,0.0000000,0
4,0.5555556,0.7777778,0.7777778,0.0000000,0.2222222,0.3333333,0.2222222,0.6666667,0
5,0.3333333,0.0000000,0.0000000,0.2222222,0.1111111,0.0000000,0.2222222,0.0000000,0
6,0.7777778,1.0000000,1.0000000,0.7777778,0.6666667,1.0000000,0.8888889,0.6666667,0


In [10]:
new_bcd$diagnosis<-bc_data_clean$diagnosis
head(new_bcd)

,Cl.thickness,Cell.size,Cell.shape,Marg.adhesion,Epith.c.size,Bare.nuclei,Bl.cromatin,Normal.nucleoli,Mitoses,diagnosis
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<fct>
1,0.4444444,0.0000000,0.0000000,0.0000000,0.1111111,0.0000000,0.2222222,0.0000000,0,benign
2,0.4444444,0.3333333,0.3333333,0.4444444,0.6666667,1.0000000,0.2222222,0.1111111,0,benign
3,0.2222222,0.0000000,0.0000000,0.0000000,0.1111111,0.1111111,0.2222222,0.0000000,0,benign
4,0.5555556,0.7777778,0.7777778,0.0000000,0.2222222,0.3333333,0.2222222,0.6666667,0,benign
5,0.3333333,0.0000000,0.0000000,0.2222222,0.1111111,0.0000000,0.2222222,0.0000000,0,benign
6,0.7777778,1.0000000,1.0000000,0.7777778,0.6666667,1.0000000,0.8888889,0.6666667,0,malignant


In [12]:
install.packages('tree')
library('tree')

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)



In [13]:
predictors <- select(new_bcd,-diagnosis)
target <- new_bcd$diagnosis

In [14]:
head(predictors)

,Cl.thickness,Cell.size,Cell.shape,Marg.adhesion,Epith.c.size,Bare.nuclei,Bl.cromatin,Normal.nucleoli,Mitoses
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,0.4444444,0.0000000,0.0000000,0.0000000,0.1111111,0.0000000,0.2222222,0.0000000,0
2,0.4444444,0.3333333,0.3333333,0.4444444,0.6666667,1.0000000,0.2222222,0.1111111,0
3,0.2222222,0.0000000,0.0000000,0.0000000,0.1111111,0.1111111,0.2222222,0.0000000,0
4,0.5555556,0.7777778,0.7777778,0.0000000,0.2222222,0.3333333,0.2222222,0.6666667,0
5,0.3333333,0.0000000,0.0000000,0.2222222,0.1111111,0.0000000,0.2222222,0.0000000,0
6,0.7777778,1.0000000,1.0000000,0.7777778,0.6666667,1.0000000,0.8888889,0.6666667,0


In [15]:
head(target)

[1] benign    benign    benign    benign    benign    malignant
Levels: benign malignant

In [16]:
# Create a training set and a testing set
set.seed(123)
train_indices <- sample(1:nrow(new_bcd), 0.7 * nrow(new_bcd))
train_data <- new_bcd[train_indices, ]
test_data <- new_bcd[-train_indices, ]

In [30]:
head(test_data)

,Cl.thickness,Cell.size,Cell.shape,Marg.adhesion,Epith.c.size,Bare.nuclei,Bl.cromatin,Normal.nucleoli,Mitoses,diagnosis
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<fct>
1,0.4444444,0.0000000,0.0000000,0.0000000,0.1111111,0.0000000,0.2222222,0.0000000,0.000,benign
2,0.4444444,0.3333333,0.3333333,0.4444444,0.6666667,1.0000000,0.2222222,0.1111111,0.000,benign
3,0.2222222,0.0000000,0.0000000,0.0000000,0.1111111,0.1111111,0.2222222,0.0000000,0.000,benign
9,0.1111111,0.0000000,0.0000000,0.0000000,0.1111111,0.0000000,0.0000000,0.0000000,0.500,benign
15,0.7777778,0.6666667,0.4444444,1.0000000,0.6666667,0.8888889,0.4444444,0.4444444,0.375,malignant
17,0.3333333,0.0000000,0.0000000,0.0000000,0.1111111,0.0000000,0.1111111,0.0000000,0.000,benign


In [19]:
# Train the decision tree model
dt_model <- tree(diagnosis ~ ., data = train_data)

In [20]:
# Print a summary of the model
summary(dt_model)


Classification tree:
tree(formula = diagnosis ~ ., data = train_data)
Variables actually used in tree construction:
[1] "Cell.size"       "Bare.nuclei"     "Cl.thickness"    "Normal.nucleoli"
[5] "Marg.adhesion"  
Number of terminal nodes:  8 
Residual mean deviance:  0.1315 = 61.83 / 470 
Misclassification error rate: 0.02929 = 14 / 478 

In [22]:
# Predict on the test set
predictions <- predict(dt_model, test_data, type = "class")

In [23]:
predictions

[1] benign    malignant benign    benign    malignant benign    benign   
  [8] malignant benign    malignant benign    malignant malignant benign   
 [15] benign    malignant benign    malignant benign    malignant benign   
 [22] benign    benign    malignant malignant benign    benign    benign   
 [29] benign    benign    malignant malignant malignant benign    benign   
 [36] malignant benign    benign    benign    benign    malignant benign   
 [43] benign    benign    malignant malignant benign    benign    malignant
 [50] malignant benign    benign    benign    malignant malignant malignant
 [57] benign    malignant malignant malignant malignant malignant benign   
 [64] malignant malignant benign    benign    malignant malignant malignant
 [71] malignant benign    benign    benign    malignant benign    benign   
 [78] malignant malignant benign    malignant malignant benign    benign   
 [85] malignant malignant benign    benign    benign    malignant benign   
 [92] benign    malignant benign    benign    benign    benign    benign   
 [99] malignant malignant benign    benign    malignant benign    benign   
[106] benign    benign    benign    benign    benign    benign    benign   
[113] malignant malignant benign    benign    malignant benign    benign   
[120] malignant benign    benign    malignant benign    benign    benign   
[127] benign    benign    malignant malignant benign    malignant benign   
[134] benign    benign    benign    malignant benign    benign    benign   
[141] benign    benign    benign    malignant benign    benign    benign   
[148] malignant benign    malignant benign    benign    benign    benign   
[155] benign    benign    benign    benign    benign    malignant benign   
[162] malignant benign    malignant benign    benign    benign    benign   
[169] benign    benign    benign    benign    malignant benign    benign   
[176] malignant benign    malignant malignant malignant benign    benign   
[183] benign    benign    benign    benign    benign    benign    benign   
[190] benign    benign    benign    malignant malignant malignant benign   
[197] benign    benign    benign    malignant benign    benign    malignant
[204] benign    benign   
Levels: benign malignant

In [24]:
confusion_matrix <- table(predictions, test_data$diagnosis)
accuracy <- sum(diag(confusion_matrix)) / sum(confusion_matrix)
print(paste("Accuracy:", accuracy))

[1] "Accuracy: 0.946341463414634"


In [36]:
custom_data <- data.frame(
  Cl.thickness = c(5.1, 4.9,0.4444444),
  Cell.size = c(3.5, 3.0,0),
  Cell.shape = c(1.4, 1.4,0),
  Marg.adhesion = c(0.2, 0.2,0),
  Epith.c.size = c(0.5,0.2,0.111111),
  Bare.nuclei = c(0.3, 0.4,0),
  Bl.cromatin = c(0,0,0.22222),
  Normal.nucleoli= c(0.9,0.5,0.000),
  Mitoses = c(0.375, 0.4,0.00)
)

In [37]:
# Testing on custom data
sample_prediction <- predict(dt_model, custom_data, type="class")

In [38]:
sample_prediction

[1] malignant malignant benign   
Levels: benign malignant